### **r/place 2023 Sentiment Analysis Model**
This Jupyter notebook will fine-tune the DistilBERT model to perform sentiment analysis on Reddit comments in July 2023. Feel free to tweak the variables and code here. Credits are included at the end of the notebook.

**Install Dependencies**<br>
This notebook has been tested on Python 3.11.2 and uses Pytorch.

In [1]:
import csv
import datasets
import pandas as pd
import sklearn
import torch

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Load the Data**<br>
The target CSV file has Reddit comments in Column 0 and a score in Column 1. The scores correspond to the following sentiments: -1 = negative, 0 = neutral, 1 = positive. We will tweak the range from [-1, 1] to [0, 2] to match the model's labels.

In [2]:
# define the data path and store the comments in a list
data_path = "data/Reddit_Data.csv"
comments_and_scores = []

# read the csv and store each comment with its respective score
with open(data_path, "r", encoding="utf8") as f:
    csv_reader = csv.reader(f)
    next(csv_reader)
    for row in csv_reader:
        comment, score = row
        comments_and_scores.append((comment, int(score)+1))

print(comments_and_scores[0])
print(len(comments_and_scores))

(' family mormon have never tried explain them they still stare puzzled from time time like some kind strange creature nonetheless they have come admire for the patience calmness equanimity acceptance and compassion have developed all the things buddhism teaches ', 2)
37249


**Separate Training and Testing Datasets**<br>
We need to separate these comments into training and testing datasets.

In [3]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(comments_and_scores,
                                       test_size=0.2,
                                       random_state=24)

In [4]:
print(len(train_set))
print(len(test_set))

29799
7450


In [5]:
print(train_set[0])
print(test_set[0])

(' yeh hindutva promote kar rahe hai ', 1)
('vote for the best available candidate nota waste like you said ', 2)


In [6]:
# extract the training comments and scores
train_comments = [group[0] for group in train_set]
train_scores = [group[1] for group in train_set]

# extract the testing comments and scores
test_comments = [group[0] for group in test_set]
test_scores = [group[1] for group in test_set]

In [7]:
print(train_comments[0], train_scores[0])
print(test_comments[0], test_scores[0])

 yeh hindutva promote kar rahe hai  1
vote for the best available candidate nota waste like you said  2


Now that we have the training and testing datasets, we will convert them into Pandas DataFrame objects.

In [8]:
# extract the train set from the list
train_set = {"text": train_comments, "labels": train_scores}
train_set = pd.DataFrame(train_set)
train_set = datasets.Dataset.from_pandas(train_set)
print(train_set)

Dataset({
    features: ['text', 'score'],
    num_rows: 29799
})


In [9]:
# extract the test set from the list
test_set = {"text": test_comments, "labels": test_scores}
test_set = pd.DataFrame(test_set)
test_set = datasets.Dataset.from_pandas(test_set)
print(test_set)

Dataset({
    features: ['text', 'score'],
    num_rows: 7450
})


**Tokenize the Data**<br>
Prior to training the model, we will tokenize the Reddit comments into small pieces to make it easier for the model to identify the comment's sentiment. Note: I disabled the warning for the fast tokenizer request as it will prevent the trainer from running the .train() function later in the notebook.

In [24]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [35]:
# prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=128)

tokenized_train = train_set.map(preprocess_function, batched=True)
tokenized_test = test_set.map(preprocess_function, batched=True)
































Map: 100%|██████████| 29799/29799 [01:17<00:00, 382.46 examples/s]









Map: 100%|██████████| 7450/7450 [00:19<00:00, 385.01 examples/s]


In [36]:
# Use data_collector to convert our samples to PyTorch tensors and concatenate them with the correct amount of padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [37]:
# Define DistilBERT as our base model:
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
# Define the evaluation metrics
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [41]:
# Define a new Trainer with all the objects we constructed so far
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='args/',
    evaluation_strategy='epoch',
    save_total_limit=2,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='logs/',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [42]:
trainer.train()

  0%|          | 0/5589 [07:11<?, ?it/s]
c:\Python311\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/5589 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.